In [ ]:
import os
import json
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Obtener API Key de OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Base de datos de vuelos
air_flights = {
    "londres": {"price": "$799", "origin": "JFK (New York)", "departure_time": "10:30 AM", "date": "2025-02-15"},
    "parís": {"price": "$899", "origin": "LAX (Los Angeles)", "departure_time": "3:00 PM", "date": "2025-02-16"},
    "tokyo": {"price": "$1400", "origin": "ORD (Chicago)", "departure_time": "6:45 PM", "date": "2025-02-17"},
    "berlín": {"price": "$499", "origin": "ATL (Atlanta)", "departure_time": "8:20 AM", "date": "2025-02-18"}
}

# Archivo JSON donde guardamos las reservas
TICKET_FILE = "reservations.json"

# Función para leer las reservas desde JSON
def load_tickets():
    if not os.path.exists(TICKET_FILE):
        return {}
    with open(TICKET_FILE, "r", encoding="utf-8") as file:
        data = json.load(file)
    print(f"[DEBUG] Cargando reservas desde JSON: {json.dumps(data, indent=4)}")
    return data

# Función para guardar una reserva en JSON
def save_ticket(user, destination):
    tickets = load_tickets()
    user = user.capitalize()
    
    if destination.lower() not in air_flights:
        return "⚠️ No hay vuelos disponibles para ese destino."
    
    flight_info = air_flights[destination.lower()]
    
    if user not in tickets:
        tickets[user] = []
    
    # Verificar si el usuario ya tiene un billete para el mismo destino
    for ticket in tickets[user]:
        if ticket["destination"] == destination.capitalize():
            return f"🔔 {user}, ya tienes un billete reservado para {destination.capitalize()}."
    
    new_ticket = {
        "destination": destination.capitalize(),
        "origin": flight_info["origin"],
        "departure_time": flight_info["departure_time"],
        "date": flight_info["date"],
        "price": flight_info["price"]
    }
    tickets[user].append(new_ticket)
    
    with open(TICKET_FILE, "w", encoding="utf-8") as file:
        json.dump(tickets, file, indent=4, ensure_ascii=False)
    
    print(f"[DEBUG] Reserva guardada en JSON para {user}: {new_ticket}")
    return (f"✅ {user}, tu reserva ha sido confirmada.\n"
            f"✈️ {new_ticket['origin']} → {new_ticket['destination']}\n"
            f"📅 {new_ticket['date']} 🕒 {new_ticket['departure_time']}\n"
            f"💰 {new_ticket['price']}")

# Función para consultar reservas por usuario
def get_user_reservations(username):
    tickets = load_tickets()
    username = username.capitalize()
    
    print(f"[DEBUG] Consultando reservas para {username}")
    
    if username not in tickets or not tickets[username]:
        print(f"[DEBUG] No se encontraron reservas para {username}")
        return f"❌ No hay reservas para {username}."
    
    response = f"📋 **Reservas de {username}:**\n"
    for ticket in tickets[username]:
        response += (f"✈️ {ticket['origin']} → {ticket['destination']}\n"
                     f"📅 {ticket['date']} 🕒 {ticket['departure_time']}\n"
                     f"💰 {ticket['price']}\n\n")
    
    print(f"[DEBUG] Reservas encontradas para {username}: {json.dumps(tickets[username], indent=4)}")
    return response.strip()

# **Función que usa OpenAI para detectar intención y responder**
def chat_with_openai(message, history):
    print(f"[DEBUG] Mensaje recibido: {message}")
    
    system_prompt = (
        "Eres un chatbot que maneja reservas de vuelos. Solo puedes realizar las siguientes acciones:\n"
        "- Mostrar los vuelos disponibles si el usuario lo solicita.\n"
        "- Reservar un vuelo cuando el usuario mencione una ciudad y su nombre.\n"
        "- Consultar las reservas de un usuario si lo solicita.\n"
        "- Saludar si el usuario te saluda.\n"
        "IMPORTANTE: Para consultar reservas, responde usando get_user_reservations(username). NO inventes respuestas."
    )

    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    bot_response = response.choices[0].message.content.lower()
    print(f"[DEBUG] Respuesta de OpenAI: {bot_response}")
    
    if "vuelos disponibles" in bot_response:
        return show_flights()
    
    if "qué reservas tiene" in bot_response:
        parts = bot_response.split("qué reservas tiene")
        if len(parts) > 1:
            username = parts[1].strip().capitalize()
            return get_user_reservations(username)
    
    return bot_response

# Función para mostrar vuelos disponibles
def show_flights():
    response = "✈️ **Vuelos disponibles:**\n"
    for city, info in air_flights.items():
        response += (f"- **{info['origin']} → {city.capitalize()}**\n"
                     f"  📅 {info['date']} 🕒 {info['departure_time']} 💰 {info['price']}\n")
    print("[DEBUG] Mostrando vuelos disponibles")
    return response

# Interfaz en Gradio
with gr.Blocks() as app:
    gr.Markdown("## ✈️ Chatbot FlightAI - Reservas de Billetes")
    chat_interface = gr.ChatInterface(fn=chat_with_openai, type="messages")
    app.launch()
